In [ ]:
#############################################베이스라인  버전######################################################

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.17 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [ ]:
#import
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from tqdm import tqdm

import warnings
warnings.filterwarnings(action='ignore')

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

#Hyperparameter Setting
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':5,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':32,
    'SEED':41
}

#Fixed RandomSeed
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

#Data Pre-processing
df = pd.read_csv('./train.csv')

train_len = int(len(df) * 0.8)
train_df = df.iloc[:train_len]
val_df = df.iloc[train_len:]

train_label_vec = train_df.iloc[:,2:].values.astype(np.float32)
val_label_vec = val_df.iloc[:,2:].values.astype(np.float32)

CFG['label_size'] = train_label_vec.shape[1]

#CustomDataset
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms

    def __getitem__(self, index):
        img_path = self.img_path_list[index]

        image = cv2.imread(img_path)

        if self.transforms is not None:
            image = self.transforms(image=image)['image']

        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image

    def __len__(self):
        return len(self.img_path_list)
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])
train_dataset = CustomDataset(train_df['path'].values, train_label_vec, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

val_dataset = CustomDataset(val_df['path'].values, val_label_vec, test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)
#Model Define
class BaseModel(nn.Module):
    def __init__(self, gene_size=CFG['label_size']):
        super(BaseModel, self).__init__()
        self.backbone = models.resnet50(pretrained=True)
        self.regressor = nn.Linear(1000, gene_size)

    def forward(self, x):
        x = self.backbone(x)
        x = self.regressor(x)
        return x
#Train
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.MSELoss().to(device)

    best_loss = 99999999
    best_model = None

    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            output = model(imgs)
            loss = criterion(output, labels)

            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

        _val_loss = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}]')

        if scheduler is not None:
            scheduler.step(_val_loss)

        if best_loss > _val_loss:
            best_loss = _val_loss
            best_model = model

    return best_mode

#valid
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)

            pred = model(imgs)

            loss = criterion(pred, labels)

            val_loss.append(loss.item())

        _val_loss = np.mean(val_loss)

    return _val_loss

#Run!!
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)


#Inference

test = pd.read_csv('./test.csv')

test_dataset = CustomDataset(test['path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(test_loader):
            imgs = imgs.to(device).float()
            pred = model(imgs)

            preds.append(pred.detach().cpu())

    preds = torch.cat(preds).numpy()

    return preds

preds = inference(infer_model, test_loader, device)

#Submission
submit = pd.read_csv('./sample_submission.csv')
submit.iloc[:, 1:] = np.array(preds).astype(np.float32)
submit.to_csv('./baseline_submit.csv', index=False)


In [ ]:
#############################################베이스라인 조각 버전 loss,tqdm######################################################

In [ ]:
#import
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from tqdm import tqdm

import warnings
warnings.filterwarnings(action='ignore')

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.17 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [ ]:
!unzip "content/h&e_db.zip"

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: train/TRAIN_1992.png    
  inflating: train/TRAIN_1993.png    
  inflating: train/TRAIN_1994.png    
  inflating: train/TRAIN_1995.png    
  inflating: train/TRAIN_1996.png    
  inflating: train/TRAIN_1997.png    
  inflating: train/TRAIN_1998.png    
  inflating: train/TRAIN_1999.png    
  inflating: train/TRAIN_2000.png    
  inflating: train/TRAIN_2001.png    
  inflating: train/TRAIN_2002.png    
  inflating: train/TRAIN_2003.png    
  inflating: train/TRAIN_2004.png    
  inflating: train/TRAIN_2005.png    
  inflating: train/TRAIN_2006.png    
  inflating: train/TRAIN_2007.png    
  inflating: train/TRAIN_2008.png    
  inflating: train/TRAIN_2009.png    
  inflating: train/TRAIN_2010.png    
  inflating: train/TRAIN_2011.png    
  inflating: train/TRAIN_2012.png    
  inflating: train/TRAIN_2013.png    
  inflating: train/TRAIN_2014.png    
  inflating: train/TRAIN_2015.png    
  inflating: train/TRAIN_2016.png    
  inflating: t

In [ ]:
#Hyperparameter Setting
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':10,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':32,
    'SEED':41
}

In [ ]:
#Fixed RandomSeed
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [ ]:
#Data Pre-processing
df = pd.read_csv('/content/train.csv')

train_len = int(len(df) * 0.8)
train_df = df.iloc[:train_len]
val_df = df.iloc[train_len:]

train_label_vec = train_df.iloc[:,2:].values.astype(np.float32)
val_label_vec = val_df.iloc[:,2:].values.astype(np.float32)

CFG['label_size'] = train_label_vec.shape[1]

In [ ]:
train_df.head(10)

,ID,path,AL645608.7,HES4,TNFRSF18,TNFRSF4,SDF4,ACAP3,INTS11,MXRA8,...,MT-ATP8,MT-ATP6,MT-CO3,MT-ND3,MT-ND4L,MT-ND4,MT-ND5,MT-CYB,BX004987.1,AL592183.1
0,TRAIN_0000,./train/TRAIN_0000.png,0.000506,0.010635,-0.000213,-0.000846,1.512467,0.021131,0.024409,0.004109,...,-0.168265,2.858198,3.154686,2.283563,0.417627,2.639062,0.809258,2.053162,-0.000464,-0.000459
1,TRAIN_0001,./train/TRAIN_0001.png,0.000506,0.010635,-0.000213,-0.000846,1.508787,0.021131,0.024409,0.004109,...,-0.168265,2.392524,3.103424,2.462341,0.552690,3.089195,0.578029,2.488428,-0.000464,-0.000459
2,TRAIN_0002,./train/TRAIN_0002.png,-0.000415,0.005658,-0.000413,0.003148,0.109204,0.013978,0.049823,0.005327,...,0.189374,2.730253,3.212037,0.377279,0.340573,3.359260,1.935161,2.502990,0.000598,0.002011
3,TRAIN_0003,./train/TRAIN_0003.png,-0.000855,0.004366,0.000684,0.000865,0.503090,0.295115,0.303922,-0.004290,...,-0.158511,2.753111,2.892158,2.218189,0.677302,2.601587,1.150156,2.198914,0.000307,0.001789
4,TRAIN_0004,./train/TRAIN_0004.png,0.000506,0.010635,-0.000213,-0.000846,0.905195,0.021131,1.597454,0.004109,...,1.097993,3.760496,3.801140,3.460122,1.641039,3.735806,1.666373,3.408605,0.820977,-0.000459
5,TRAIN_0005,./train/TRAIN_0005.png,-0.000855,0.004366,0.000684,0.000865,-0.027202,-0.004779,0.625543,-0.004290,...,-0.158510,2.811021,2.913606,1.905480,-0.197689,2.652742,0.437270,2.056819,0.000307,0.001789
6,TRAIN_0006,./train/TRAIN_0006.png,-0.000855,0.004366,0.000684,0.000865,-0.027202,-0.004779,0.495353,-0.004290,...,-0.158510,2.604077,2.781954,2.130168,0.293635,2.710687,1.079293,2.177899,0.000307,0.001789
7,TRAIN_0007,./train/TRAIN_0007.png,-0.000855,0.004366,0.000684,0.000865,0.664473,-0.004779,0.004029,-0.004290,...,0.128436,2.667228,3.052634,2.033990,0.493985,2.384952,0.661368,2.312178,0.000307,0.001789
8,TRAIN_0008,./train/TRAIN_0008.png,0.000506,0.010635,-0.000213,-0.000846,0.083756,0.021131,0.024409,0.004109,...,1.244236,3.505259,3.966054,3.533652,1.246053,3.982099,2.456003,3.559738,-0.000464,-0.000459
9,TRAIN_0009,./train/TRAIN_0009.png,0.000506,0.010635,-0.000213,-0.000846,0.809897,0.021131,0.024409,0.004109,...,-0.168265,3.420015,3.368411,3.083411,1.835379,3.220352,1.001258,3.154074,-0.000464,-0.000459


In [ ]:
#CustomDataset
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms

    def __getitem__(self, index):
        img_path = self.img_path_list[index]

        image = cv2.imread(img_path)

        if self.transforms is not None:
            image = self.transforms(image=image)['image']

        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image

    def __len__(self):
        return len(self.img_path_list)
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])
train_dataset = CustomDataset(train_df['path'].values, train_label_vec, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

val_dataset = CustomDataset(val_df['path'].values, val_label_vec, test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
"""
ResNet 계열: resnet18, resnet34, resnet50, resnet101, resnet152 #이거  o
VGG 계열: vgg11, vgg13, vgg16, vgg19 등과 BatchNorm 버전 #이거 o
DenseNet 계열: densenet121, densenet169, densenet201, densenet161 #이거 o
Inception 계열: inception_v3 #이거
AlexNet: alexnet
SqueezeNet: squeezenet1_0, squeezenet1_1 # 이거
MobileNet: mobilenet_v2, mobilenet_v3_large, mobilenet_v3_small #이거
ShuffleNet: shufflenet_v2_x0_5, shufflenet_v2_x1_0, shufflenet_v2_x1_5, shufflenet_v2_x2_0
EfficientNet: efficientnet_b0, efficientnet_b1, ..., efficientnet_b7
RegNet: regnet_y_400mf, regnet_y_800mf, ..., regnet_y_32gf, regnet_x_400mf, ..., regnet_x_32gf
GoogLeNet: googlenet
MNASNet: mnasnet0_5, mnasnet0_75, mnasnet1_0, mnasnet1_3
Wide ResNet: wide_resnet50_2, wide_resnet101_2
ResNeXt: resnext50_32x4d, resnext101_32x8d
"""

In [ ]:
#Model Define
class BaseModel(nn.Module):
    def __init__(self, gene_size=CFG['label_size']):
        super(BaseModel, self).__init__()
        self.backbone = models.resnet50(pretrained=True)
        self.regressor = nn.Linear(1000, gene_size)
        print("r50")

    def forward(self, x):
        x = self.backbone(x)
        x = self.regressor(x)
        return x

In [ ]:
!pip install tqdm

In [ ]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.MSELoss().to(device)

    best_loss = float('inf')
    best_model = None

    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = 0.0
        total_samples = 0

        train_bar = tqdm(train_loader, desc=f"Epoch {epoch} Training", leave=False)
        for imgs, labels in train_bar:
            imgs = imgs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(imgs)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            batch_size = imgs.size(0)
            train_loss += loss.item() * batch_size
            total_samples += batch_size

            train_bar.set_postfix(loss=loss.item())

        avg_train_loss = train_loss / total_samples

        avg_val_loss = validation(model, criterion, val_loader, device, epoch)

        print(f'Epoch [{epoch}], Train Loss: [{avg_train_loss:.5f}], Val Loss: [{avg_val_loss:.5f}]')

        if scheduler is not None:
            scheduler.step(avg_val_loss)

        if best_loss > avg_val_loss:
            best_loss = avg_val_loss
            best_model = model.state_dict()

    model.load_state_dict(best_model)
    return model

def validation(model, criterion, val_loader, device, epoch):
    model.eval()
    val_loss = 0.0
    total_samples = 0

    val_bar = tqdm(val_loader, desc=f"Epoch {epoch} Validation", leave=False)
    with torch.no_grad():
        for imgs, labels in val_bar:
            imgs = imgs.to(device)
            labels = labels.to(device)

            outputs = model(imgs)
            loss = criterion(outputs, labels)

            batch_size = imgs.size(0)
            val_loss += loss.item() * batch_size
            total_samples += batch_size

            val_bar.set_postfix(loss=loss.item())

    avg_val_loss = val_loss / total_samples
    return avg_val_loss


In [ ]:
#Run!!
model = BaseModel()
#print(model)
model.eval()
optimizer = torch.optim.AdamW(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 157MB/s]


r50


Epoch [1], Train Loss: [0.06520], Val Loss: [0.04821]


Epoch [2], Train Loss: [0.04801], Val Loss: [0.04737]


Epoch [3], Train Loss: [0.04717], Val Loss: [0.04714]


...

Epoch [6], Train Loss: [0.04578], Val Loss: [0.04773]


Epoch [7], Train Loss: [0.04544], Val Loss: [0.04756]


Epoch [8], Train Loss: [0.04526], Val Loss: [0.05079]


Epoch [9], Train Loss: [0.04476], Val Loss: [0.04593]


Epoch [10], Train Loss: [0.04449], Val Loss: [0.04598]


In [ ]:
#Inference

test = pd.read_csv('/content/test.csv')

test_dataset = CustomDataset(test['path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(test_loader):
            imgs = imgs.to(device).float()
            pred = model(imgs)

            preds.append(pred.detach().cpu())

    preds = torch.cat(preds).numpy()

    return preds

preds = inference(infer_model, test_loader, device)

100%|██████████| 72/72 [00:08<00:00,  8.95it/s]


In [ ]:
#Submission
submit = pd.read_csv('/content/sample_submission.csv')
submit.iloc[:, 1:] = np.array(preds).astype(np.float32)
submit.to_csv('/content/no_pro_res_10e.csv', index=False)

In [ ]:
#############################################로그 변환 버전######################################################

In [ ]:
# import
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from tqdm import tqdm

import warnings
warnings.filterwarnings(action='ignore')

# 디바이스 설정
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Hyperparameter Setting
CFG = {
    'IMG_SIZE': 224,
    'EPOCHS': 10,
    'LEARNING_RATE': 3e-4,
    'BATCH_SIZE': 32,
    'SEED': 41
}

# Fixed RandomSeed
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED'])  # Seed 고정

In [ ]:

# Data Pre-processing
df = pd.read_csv('/content/train.csv')

train_len = int(len(df) * 0.8)
train_df = df.iloc[:train_len].reset_index(drop=True)
val_df = df.iloc[train_len:].reset_index(drop=True)

train_label_vec = train_df.iloc[:, 2:].values.astype(np.float32)
val_label_vec = val_df.iloc[:, 2:].values.astype(np.float32)

# 라벨 값을 로그 스케일로 변환
train_label_vec = np.log1p(train_label_vec)
val_label_vec = np.log1p(val_label_vec)

CFG['label_size'] = train_label_vec.shape[1]

# CustomDataset
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list=None, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms

    def __getitem__(self, index):
        img_path = self.img_path_list[index]

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transforms is not None:
            image = self.transforms(image=image)['image']

        if self.label_list is not None:
            label = self.label_list[index]
            label = torch.tensor(label, dtype=torch.float32)
            return image, label
        else:
            return image

    def __len__(self):
        return len(self.img_path_list)

# 데이터 변환 설정
train_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

test_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

# 데이터셋 및 데이터로더 생성
train_dataset = CustomDataset(train_df['path'].values, train_label_vec, train_transform)
train_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val_df['path'].values, val_label_vec, test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)



In [ ]:
"""
ResNet 계열: resnet18, resnet34, resnet50, resnet101, resnet152
VGG 계열: vgg11, vgg13, vgg16, vgg19 등과 BatchNorm 버전
DenseNet 계열: densenet121, densenet169, densenet201, densenet161
Inception 계열: inception_v3
AlexNet: alexnet
SqueezeNet: squeezenet1_0, squeezenet1_1
MobileNet: mobilenet_v2, mobilenet_v3_large, mobilenet_v3_small
ShuffleNet: shufflenet_v2_x0_5, shufflenet_v2_x1_0, shufflenet_v2_x1_5, shufflenet_v2_x2_0
EfficientNet: efficientnet_b0, efficientnet_b1, ..., efficientnet_b7
RegNet: regnet_y_400mf, regnet_y_800mf, ..., regnet_y_32gf, regnet_x_400mf, ..., regnet_x_32gf
GoogLeNet: googlenet
MNASNet: mnasnet0_5, mnasnet0_75, mnasnet1_0, mnasnet1_3
Wide ResNet: wide_resnet50_2, wide_resnet101_2
ResNeXt: resnext50_32x4d, resnext101_32x8d
"""

'\nResNet 계열: resnet18, resnet34, resnet50, resnet101, resnet152\nVGG 계열: vgg11, vgg13, vgg16, vgg19 등과 BatchNorm 버전\nDenseNet 계열: densenet121, densenet169, densenet201, densenet161\nInception 계열: inception_v3\nAlexNet: alexnet\nSqueezeNet: squeezenet1_0, squeezenet1_1\nMobileNet: mobilenet_v2, mobilenet_v3_large, mobilenet_v3_small\nShuffleNet: shufflenet_v2_x0_5, shufflenet_v2_x1_0, shufflenet_v2_x1_5, shufflenet_v2_x2_0\nEfficientNet: efficientnet_b0, efficientnet_b1, ..., efficientnet_b7\nRegNet: regnet_y_400mf, regnet_y_800mf, ..., regnet_y_32gf, regnet_x_400mf, ..., regnet_x_32gf\nGoogLeNet: googlenet\nMNASNet: mnasnet0_5, mnasnet0_75, mnasnet1_0, mnasnet1_3\nWide ResNet: wide_resnet50_2, wide_resnet101_2\nResNeXt: resnext50_32x4d, resnext101_32x8d\n'

In [ ]:
# Model Define
class BaseModel(nn.Module):
    def __init__(self, gene_size=CFG['label_size']):
        super(BaseModel, self).__init__()
        self.backbone = models.resnet50(pretrained=True)
        num_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Identity()  # 기존의 fc 레이어를 제거
        self.regressor = nn.Linear(num_features, gene_size)

    def forward(self, x):
        x = self.backbone(x)
        x = self.regressor(x)
        return x


In [ ]:
# 학습 및 검증 함수 정의
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.MSELoss().to(device)

    best_loss = float('inf')
    best_model = None

    for epoch in range(1, CFG['EPOCHS'] + 1):
        model.train()
        train_loss = 0.0
        total_samples = 0

        train_bar = tqdm(train_loader, desc=f"Epoch {epoch} Training", leave=False)
        for imgs, labels in train_bar:
            imgs = imgs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(imgs)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            batch_size = imgs.size(0)
            train_loss += loss.item() * batch_size
            total_samples += batch_size

            train_bar.set_postfix(loss=loss.item())

        avg_train_loss = train_loss / total_samples

        avg_val_loss = validation(model, criterion, val_loader, device, epoch)

        print(f'Epoch [{epoch}], Train Loss: [{avg_train_loss:.5f}], Val Loss: [{avg_val_loss:.5f}]')

        if scheduler is not None:
            scheduler.step(avg_val_loss)

        if best_loss > avg_val_loss:
            best_loss = avg_val_loss
            best_model = model.state_dict()

    model.load_state_dict(best_model)
    return model

def validation(model, criterion, val_loader, device, epoch):
    model.eval()
    val_loss = 0.0
    total_samples = 0

    val_bar = tqdm(val_loader, desc=f"Epoch {epoch} Validation", leave=False)
    with torch.no_grad():
        for imgs, labels in val_bar:
            imgs = imgs.to(device)
            labels = labels.to(device)

            outputs = model(imgs)
            loss = criterion(outputs, labels)

            batch_size = imgs.size(0)
            val_loss += loss.item() * batch_size
            total_samples += batch_size

            val_bar.set_postfix(loss=loss.item())

    avg_val_loss = val_loss / total_samples
    return avg_val_loss


In [ ]:
# Run!!
model = BaseModel()
model.to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2,
                                                       threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

Epoch [1], Train Loss: [0.02298], Val Loss: [0.02039]


Epoch [2], Train Loss: [0.02024], Val Loss: [0.02009]


Epoch [3], Train Loss: [0.02015], Val Loss: [0.02938]


Epoch [4], Train Loss: [0.02022], Val Loss: [0.02291]


Epoch [5], Train Loss: [0.02004], Val Loss: [0.02021]


Epoch [6], Train Loss: [0.01988], Val Loss: [0.02059]


Epoch [7], Train Loss: [0.01986], Val Loss: [0.01975]


Epoch [8], Train Loss: [0.01985], Val Loss: [0.01980]


Epoch [9], Train Loss: [0.01983], Val Loss: [0.02000]


Epoch [10], Train Loss: [0.01983], Val Loss: [0.01977]


In [ ]:
# Inference
test = pd.read_csv('/content/test.csv')

test_dataset = CustomDataset(test['path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(test_loader):
            imgs = imgs.to(device)
            outputs = model(imgs)
            preds.append(outputs.cpu())

    preds = torch.cat(preds).numpy()

    # 예측값에 역로그 변환 적용
    preds = np.expm1(preds)

    return preds

preds = inference(infer_model, test_loader, device)

100%|██████████| 72/72 [00:07<00:00,  9.17it/s]


In [ ]:
#Submission
submit = pd.read_csv('/content/sample_submission.csv')
submit.iloc[:, 1:] = np.array(preds).astype(np.float32)
submit.to_csv('/content/pro_res_10e.csv', index=False)

In [ ]:
#############################################여기부터는 기본 로그변환######################################################

In [ ]:
#############################################vit 이런식######################################################

In [ ]:
# Model Define
class BaseModel(nn.Module):
    def __init__(self, gene_size=CFG['label_size']):
        super(BaseModel, self).__init__()
        # ViT 모델 로드 (ImageNet-21k로 사전 학습된 가중치 사용)
        self.backbone = timm.create_model('vit_base_patch16_224', pretrained=True)
        # 분류 헤드를 제거하고 특징 추출기만 사용
        self.backbone.head = nn.Identity()
        # 백본의 출력 특징 수
        num_features = self.backbone.num_features
        # 회귀를 위한 새로운 출력층 추가
        self.regressor = nn.Linear(num_features, gene_size)

    def forward(self, x):
        x = self.backbone(x)
        x = self.regressor(x)
        return x

In [ ]:
#############################################전체 데이터셋 사용 train 수정 버전######################################################

In [ ]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 41.6 MB/s eta 0:00:00


In [ ]:
# import
import random
import pandas as pd
import numpy as np
import os
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import timm

from tqdm import tqdm

import warnings
warnings.filterwarnings(action='ignore')

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Hyperparameter Setting
CFG = {
    'IMG_SIZE': 224,
    'EPOCHS': 10,
    'LEARNING_RATE': 3e-4,
    'BATCH_SIZE': 32,
    'SEED': 41
}

# Fixed RandomSeed
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED'])  # Seed 고정

# Data Pre-processing
df = pd.read_csv('./train.csv')

# 전체 데이터를 학습 데이터로 사용
train_df = df.reset_index(drop=True)

train_label_vec = train_df.iloc[:, 2:].values.astype(np.float32)

# 라벨 값을 로그 스케일로 변환
train_label_vec = np.log1p(train_label_vec)

CFG['label_size'] = train_label_vec.shape[1]

# CustomDataset
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list=None, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms

    def __getitem__(self, index):
        img_path = self.img_path_list[index]

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        else:
            # 기본 변환: Tensor 변환
            image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1) / 255.0  # [C, H, W]

        if self.label_list is not None:
            label = self.label_list[index]
            label = torch.tensor(label, dtype=torch.float32)
            return image, label
        else:
            return image

    def __len__(self):
        return len(self.img_path_list)

# 데이터 변환 설정 (증강 제거)
train_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

test_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

train_dataset = CustomDataset(train_df['path'].values, train_label_vec, train_transform)
train_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

# 검증 데이터셋 및 DataLoader 제거

# Model Define
# Model Define
class BaseModel(nn.Module):
    def __init__(self, gene_size=CFG['label_size']):
        super(BaseModel, self).__init__()
        self.backbone = models.resnet50(pretrained=True)
        num_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Identity()  # 기존의 fc 레이어를 제거
        self.regressor = nn.Linear(num_features, gene_size)

    def forward(self, x):
        x = self.backbone(x)
        x = self.regressor(x)
        return x

# Train 함수 정의
def train(model, optimizer, train_loader, device, scheduler=None):
    model.to(device)
    # 손실 함수를 MSELoss로 설정
    criterion = nn.MSELoss().to(device)

    best_loss = float('inf')
    best_model = None

    for epoch in range(1, CFG['EPOCHS'] + 1):
        model.train()
        train_loss = 0.0
        total_batches = 0

        train_bar = tqdm(train_loader, desc=f"Epoch {epoch} Training", leave=False)
        for imgs, labels in train_bar:
            imgs = imgs.float().to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            output = model(imgs)
            loss = criterion(output, labels)

            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            total_batches += 1

            train_bar.set_postfix(loss=loss.item())

        avg_train_loss = train_loss / total_batches
        print(f'Epoch [{epoch}], Train Loss: [{avg_train_loss:.5f}]')

        # StepLR 스케줄러 업데이트
        if scheduler is not None:
            scheduler.step()

        # 가장 낮은 손실을 가진 모델 저장 (선택 사항)
        if avg_train_loss < best_loss:
            best_loss = avg_train_loss
            best_model = model.state_dict()

    if best_model is not None:
        model.load_state_dict(best_model)
    return model

# Inference 함수 정의
def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(test_loader, desc="Inference"):
            imgs = imgs.float().to(device)
            pred = model(imgs)
            preds.append(pred.detach().cpu())

    preds = torch.cat(preds).numpy()
    # 예측값에 역로그 변환 적용
    preds = np.expm1(preds)
    return preds

# Run!! 학습
model = BaseModel()
model.to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=CFG["LEARNING_RATE"])

# 스케줄러 설정 (옵션: StepLR 사용)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.5)

infer_model = train(model, optimizer, train_loader, device, scheduler)

# Inference
test = pd.read_csv('./test.csv')

test_dataset = CustomDataset(test['path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

preds = inference(infer_model, test_loader, device)

# Submission
submit = pd.read_csv('./sample_submission.csv')
submit.iloc[:, 1:] = preds.astype(np.float32)
submit.to_csv('./baseline_submit.csv', index=False)


Epoch [1], Train Loss: [0.02241]


Epoch [2], Train Loss: [0.02013]


Epoch [3], Train Loss: [0.02003]


Epoch [4], Train Loss: [0.01988]


Epoch [5], Train Loss: [0.01971]


...

Epoch [7], Train Loss: [0.01947]


Epoch [8], Train Loss: [0.01943]


Epoch [9], Train Loss: [0.01936]


Epoch [10], Train Loss: [0.01934]


Inference: 100%|██████████| 72/72 [00:07<00:00,  9.15it/s]


In [ ]:
#Submission
submit = pd.read_csv('/content/sample_submission.csv')
submit.iloc[:, 1:] = np.array(preds).astype(np.float32)
submit.to_csv('/content/allt_pro_res_10e.csv', index=False)

In [ ]:
#############################################모델->(증강) 수정 버전######################################################

In [ ]:
# import
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
# import timm 라이브러리 추가
import timm

from tqdm import tqdm

import warnings
warnings.filterwarnings(action='ignore')

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Hyperparameter Setting
CFG = {
    'IMG_SIZE': 224,
    'EPOCHS': 5,
    'LEARNING_RATE': 3e-4,
    'BATCH_SIZE': 32,
    'SEED': 41
}

# Fixed RandomSeed
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED'])  # Seed 고정

# Data Pre-processing
df = pd.read_csv('./train.csv')

train_len = int(len(df) * 0.8)
train_df = df.iloc[:train_len].reset_index(drop=True)
val_df = df.iloc[train_len:].reset_index(drop=True)

train_label_vec = train_df.iloc[:, 2:].values.astype(np.float32)
val_label_vec = val_df.iloc[:, 2:].values.astype(np.float32)

# 라벨 값을 로그 스케일로 변환
train_label_vec = np.log1p(train_label_vec)
val_label_vec = np.log1p(val_label_vec)

CFG['label_size'] = train_label_vec.shape[1]

# CustomDataset
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list=None, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms

    def __getitem__(self, index):
        img_path = self.img_path_list[index]

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transforms is not None:
            image = self.transforms(image=image)['image']

        if self.label_list is not None:
            label = self.label_list[index]
            label = torch.tensor(label, dtype=torch.float32)
            return image, label
        else:
            return image

    def __len__(self):
        return len(self.img_path_list)

# 데이터 증강 설정 (crop 제외, rotation, saturation, flip, brightness 적용)
train_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.Rotate(limit=45, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

test_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

train_dataset = CustomDataset(train_df['path'].values, train_label_vec, train_transform)
train_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val_df['path'].values, val_label_vec, test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

# Model Define
class BaseModel(nn.Module):
    def __init__(self, gene_size=CFG['label_size']):
        super(BaseModel, self).__init__()
        # ViT 모델 로드 (ImageNet-21k로 사전 학습된 가중치 사용)

        #self.backbone = timm.create_model('vit_base_patch16_224', pretrained=True)
        self.backbone = timm.create_model('vit_base_patch16_224_in21k', pretrained=True)
        # 분류 헤드를 제거하고 특징 추출기만 사용
        self.backbone.head = nn.Identity()
        # 백본의 출력 특징 수
        num_features = self.backbone.num_features
        # 회귀를 위한 새로운 출력층 추가
        self.regressor = nn.Linear(num_features, gene_size)

    def forward(self, x):
        x = self.backbone(x)
        x = self.regressor(x)
        return x

# Train
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.MSELoss().to(device)

    best_loss = float('inf')
    best_model = None

    for epoch in range(1, CFG['EPOCHS'] + 1):
        model.train()
        train_loss = 0.0
        total_samples = 0

        train_bar = tqdm(train_loader, desc=f"Epoch {epoch} Training", leave=False)
        for imgs, labels in train_bar:
            imgs = imgs.float().to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            output = model(imgs)
            loss = criterion(output, labels)

            loss.backward()
            optimizer.step()

            batch_size = imgs.size(0)
            train_loss += loss.item() * batch_size
            total_samples += batch_size

            train_bar.set_postfix(loss=loss.item())

        avg_train_loss = train_loss / total_samples
        avg_val_loss = validation(model, criterion, val_loader, device, epoch)

        print(f'Epoch [{epoch}], Train Loss: [{avg_train_loss:.5f}], Val Loss: [{avg_val_loss:.5f}]')

        if scheduler is not None:
            scheduler.step(avg_val_loss)

        if best_loss > avg_val_loss:
            best_loss = avg_val_loss
            best_model = model.state_dict()

    model.load_state_dict(best_model)
    return model

# valid
def validation(model, criterion, val_loader, device, epoch):
    model.eval()
    val_loss = 0.0
    total_samples = 0

    val_bar = tqdm(val_loader, desc=f"Epoch {epoch} Validation", leave=False)
    with torch.no_grad():
        for imgs, labels in val_bar:
            imgs = imgs.float().to(device)
            labels = labels.to(device)

            pred = model(imgs)
            loss = criterion(pred, labels)

            batch_size = imgs.size(0)
            val_loss += loss.item() * batch_size
            total_samples += batch_size

            val_bar.set_postfix(loss=loss.item())

    avg_val_loss = val_loss / total_samples
    return avg_val_loss

# Run!!
model = BaseModel()
model.to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2,
                                                       threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

# Inference
test = pd.read_csv('./test.csv')

test_dataset = CustomDataset(test['path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(test_loader):
            imgs = imgs.to(device).float()
            pred = model(imgs)
            preds.append(pred.detach().cpu())

    preds = torch.cat(preds).numpy()
    # 예측값에 역로그 변환 적용
    preds = np.expm1(preds)
    return preds

preds = inference(infer_model, test_loader, device)



model.safetensors:   0%|          | 0.00/410M [00:00<?, ?B/s]

Epoch [1], Train Loss: [0.09845], Val Loss: [0.01998]


Epoch [2], Train Loss: [0.02015], Val Loss: [0.02001]


Epoch [3], Train Loss: [0.02017], Val Loss: [0.02007]


Epoch [4], Train Loss: [0.02019], Val Loss: [0.02008]


Epoch [5], Train Loss: [0.02008], Val Loss: [0.01997]


100%|██████████| 72/72 [00:14<00:00,  4.83it/s]


In [ ]:
#Submission
submit = pd.read_csv('/content/sample_submission.csv')
submit.iloc[:, 1:] = np.array(preds).astype(np.float32)
submit.to_csv('/content/pro_res_10e.csv', index=False)

In [ ]:
#############################################모델->(loss == huber) 수정 버전######################################################

In [ ]:
# import
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
# import timm 라이브러리 추가
import timm

from tqdm import tqdm

import warnings
warnings.filterwarnings(action='ignore')

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Hyperparameter Setting
CFG = {
    'IMG_SIZE': 224,
    'EPOCHS': 5,
    'LEARNING_RATE': 3e-4,
    'BATCH_SIZE': 32,
    'SEED': 41
}

# Fixed RandomSeed
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED'])  # Seed 고정

# Data Pre-processing
df = pd.read_csv('./train.csv')

train_len = int(len(df) * 0.8)
train_df = df.iloc[:train_len].reset_index(drop=True)
val_df = df.iloc[train_len:].reset_index(drop=True)

train_label_vec = train_df.iloc[:, 2:].values.astype(np.float32)
val_label_vec = val_df.iloc[:, 2:].values.astype(np.float32)

# 라벨 값을 로그 스케일로 변환
train_label_vec = np.log1p(train_label_vec)
val_label_vec = np.log1p(val_label_vec)

CFG['label_size'] = train_label_vec.shape[1]

# CustomDataset
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list=None, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms

    def __getitem__(self, index):
        img_path = self.img_path_list[index]

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transforms is not None:
            image = self.transforms(image=image)['image']

        if self.label_list is not None:
            label = self.label_list[index]
            label = torch.tensor(label, dtype=torch.float32)
            return image, label
        else:
            return image

    def __len__(self):
        return len(self.img_path_list)

train_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

test_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

train_dataset = CustomDataset(train_df['path'].values, train_label_vec, train_transform)
train_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val_df['path'].values, val_label_vec, test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

# Model Define
class BaseModel(nn.Module):
    def __init__(self, gene_size=CFG['label_size']):
        super(BaseModel, self).__init__()
        # ViT 모델 로드 (ImageNet-21k로 사전 학습된 가중치 사용)
        self.backbone = timm.create_model('vit_base_patch16_224', pretrained=True)
        # 분류 헤드를 제거하고 특징 추출기만 사용
        self.backbone.head = nn.Identity()
        # 백본의 출력 특징 수
        num_features = self.backbone.num_features
        # 회귀를 위한 새로운 출력층 추가
        self.regressor = nn.Linear(num_features, gene_size)

    def forward(self, x):
        x = self.backbone(x)
        x = self.regressor(x)
        return x

# Train
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    # criterion을 SmoothL1Loss로 변경
    criterion = nn.SmoothL1Loss().to(device)

    best_loss = float('inf')
    best_model = None

    for epoch in range(1, CFG['EPOCHS'] + 1):
        model.train()
        train_loss = 0.0
        total_samples = 0

        train_bar = tqdm(train_loader, desc=f"Epoch {epoch} Training", leave=False)
        for imgs, labels in train_bar:
            imgs = imgs.float().to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            output = model(imgs)
            loss = criterion(output, labels)

            loss.backward()
            optimizer.step()

            batch_size = imgs.size(0)
            train_loss += loss.item() * batch_size
            total_samples += batch_size

            train_bar.set_postfix(loss=loss.item())

        avg_train_loss = train_loss / total_samples
        avg_val_loss = validation(model, criterion, val_loader, device, epoch)

        print(f'Epoch [{epoch}], Train Loss: [{avg_train_loss:.5f}], Val Loss: [{avg_val_loss:.5f}]')

        if scheduler is not None:
            scheduler.step(avg_val_loss)

        if best_loss > avg_val_loss:
            best_loss = avg_val_loss
            best_model = model.state_dict()

    model.load_state_dict(best_model)
    return model

# valid
def validation(model, criterion, val_loader, device, epoch):
    model.eval()
    val_loss = 0.0
    total_samples = 0

    val_bar = tqdm(val_loader, desc=f"Epoch {epoch} Validation", leave=False)
    with torch.no_grad():
        for imgs, labels in val_bar:
            imgs = imgs.float().to(device)
            labels = labels.to(device)

            pred = model(imgs)
            loss = criterion(pred, labels)

            batch_size = imgs.size(0)
            val_loss += loss.item() * batch_size
            total_samples += batch_size

            val_bar.set_postfix(loss=loss.item())

    avg_val_loss = val_loss / total_samples
    return avg_val_loss

# Run!!
model = BaseModel()
model.to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2,
                                                       threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

# Inference
test = pd.read_csv('./test.csv')

test_dataset = CustomDataset(test['path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(test_loader):
            imgs = imgs.to(device).float()
            pred = model(imgs)
            preds.append(pred.detach().cpu())

    preds = torch.cat(preds).numpy()
    # 예측값에 역로그 변환 적용
    preds = np.expm1(preds)
    return preds

preds = inference(infer_model, test_loader, device)

# Submission
submit = pd.read_csv('./sample_submission.csv')
submit.iloc[:, 1:] = np.array(preds).astype(np.float32)
submit.to_csv('./baseline_submit.csv', index=False)


Epoch [1], Train Loss: [0.01442], Val Loss: [0.01005]


Epoch [2], Train Loss: [0.01010], Val Loss: [0.01008]


Epoch [3], Train Loss: [0.01011], Val Loss: [0.01005]


Epoch [4], Train Loss: [0.01011], Val Loss: [0.01004]


Epoch [5], Train Loss: [0.01003], Val Loss: [0.00998]


100%|██████████| 72/72 [00:14<00:00,  4.82it/s]


In [ ]:
#############################################모델->(loss == log cosh) 수정 버전######################################################

In [ ]:
# import
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
# import timm 라이브러리 추가
import timm

from tqdm import tqdm

import warnings
warnings.filterwarnings(action='ignore')

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Hyperparameter Setting
CFG = {
    'IMG_SIZE': 224,
    'EPOCHS': 5,
    'LEARNING_RATE': 3e-4,
    'BATCH_SIZE': 32,
    'SEED': 41
}

# Fixed RandomSeed
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # torch.backends.cudnn.deterministic = True  # 이 줄은 주석 처리하거나 삭제해도 됩니다.
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED'])  # Seed 고정

# Data Pre-processing
df = pd.read_csv('./train.csv')

train_len = int(len(df) * 0.8)
train_df = df.iloc[:train_len].reset_index(drop=True)
val_df = df.iloc[train_len:].reset_index(drop=True)

train_label_vec = train_df.iloc[:, 2:].values.astype(np.float32)
val_label_vec = val_df.iloc[:, 2:].values.astype(np.float32)

# 라벨 값을 로그 스케일로 변환
train_label_vec = np.log1p(train_label_vec)
val_label_vec = np.log1p(val_label_vec)

CFG['label_size'] = train_label_vec.shape[1]

# CustomDataset
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list=None, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms

    def __getitem__(self, index):
        img_path = self.img_path_list[index]

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transforms is not None:
            image = self.transforms(image=image)['image']

        if self.label_list is not None:
            label = self.label_list[index]
            label = torch.tensor(label, dtype=torch.float32)
            return image, label
        else:
            return image

    def __len__(self):
        return len(self.img_path_list)

train_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

test_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

train_dataset = CustomDataset(train_df['path'].values, train_label_vec, train_transform)
train_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val_df['path'].values, val_label_vec, test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

# Model Define
class BaseModel(nn.Module):
    def __init__(self, gene_size=CFG['label_size']):
        super(BaseModel, self).__init__()
        # ViT 모델 로드 (ImageNet-21k로 사전 학습된 가중치 사용)
        self.backbone = timm.create_model('vit_base_patch16_224', pretrained=True)
        # 분류 헤드를 제거하고 특징 추출기만 사용
        self.backbone.head = nn.Identity()
        # 백본의 출력 특징 수
        num_features = self.backbone.num_features
        # 회귀를 위한 새로운 출력층 추가
        self.regressor = nn.Linear(num_features, gene_size)

    def forward(self, x):
        x = self.backbone(x)
        x = self.regressor(x)
        return x

# Log-Cosh Loss 정의
class LogCoshLoss(nn.Module):
    def __init__(self):
        super(LogCoshLoss, self).__init__()

    def forward(self, y_pred, y_true):
        loss = torch.log(torch.cosh(y_pred - y_true + 1e-12))
        return torch.mean(loss)

# Train
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    # criterion을 LogCoshLoss로 변경
    criterion = LogCoshLoss().to(device)

    best_loss = float('inf')
    best_model = None

    for epoch in range(1, CFG['EPOCHS'] + 1):
        model.train()
        train_loss = 0.0
        total_samples = 0

        train_bar = tqdm(train_loader, desc=f"Epoch {epoch} Training", leave=False)
        for imgs, labels in train_bar:
            imgs = imgs.float().to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            output = model(imgs)
            loss = criterion(output, labels)

            loss.backward()
            optimizer.step()

            batch_size = imgs.size(0)
            train_loss += loss.item() * batch_size
            total_samples += batch_size

            train_bar.set_postfix(loss=loss.item())

        avg_train_loss = train_loss / total_samples
        avg_val_loss = validation(model, criterion, val_loader, device, epoch)

        print(f'Epoch [{epoch}], Train Loss: [{avg_train_loss:.5f}], Val Loss: [{avg_val_loss:.5f}]')

        if scheduler is not None:
            scheduler.step(avg_val_loss)

        if best_loss > avg_val_loss:
            best_loss = avg_val_loss
            best_model = model.state_dict()

    model.load_state_dict(best_model)
    return model

# valid
def validation(model, criterion, val_loader, device, epoch):
    model.eval()
    val_loss = 0.0
    total_samples = 0

    val_bar = tqdm(val_loader, desc=f"Epoch {epoch} Validation", leave=False)
    with torch.no_grad():
        for imgs, labels in val_bar:
            imgs = imgs.float().to(device)
            labels = labels.to(device)

            pred = model(imgs)
            loss = criterion(pred, labels)

            batch_size = imgs.size(0)
            val_loss += loss.item() * batch_size
            total_samples += batch_size

            val_bar.set_postfix(loss=loss.item())

    avg_val_loss = val_loss / total_samples
    return avg_val_loss

# Run!!
model = BaseModel()
model.to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2,
                                                       threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

# Inference
test = pd.read_csv('./test.csv')

test_dataset = CustomDataset(test['path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(test_loader):
            imgs = imgs.to(device).float()
            pred = model(imgs)
            preds.append(pred.detach().cpu())

    preds = torch.cat(preds).numpy()
    # 예측값에 역로그 변환 적용
    preds = np.expm1(preds)
    return preds

preds = inference(infer_model, test_loader, device)

# Submission
submit = pd.read_csv('./sample_submission.csv')
submit.iloc[:, 1:] = np.array(preds).astype(np.float32)
submit.to_csv('./baseline_submit.csv', index=False)


Epoch [1], Train Loss: [0.01362], Val Loss: [0.00965]


Epoch [2], Train Loss: [0.00969], Val Loss: [0.00968]


Epoch [3], Train Loss: [0.00970], Val Loss: [0.00965]


Epoch [4], Train Loss: [0.00970], Val Loss: [0.00964]


Epoch [5], Train Loss: [0.00963], Val Loss: [0.00958]


100%|██████████| 72/72 [00:14<00:00,  4.80it/s]


In [ ]:
#############################################모델->(loss == huber) 수정 버전######################################################

In [ ]:
# import
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
# import timm 라이브러리 추가
import timm

from tqdm import tqdm

import warnings
warnings.filterwarnings(action='ignore')

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Hyperparameter Setting
CFG = {
    'IMG_SIZE': 224,
    'EPOCHS': 5,
    'LEARNING_RATE': 3e-4,
    'BATCH_SIZE': 32,
    'SEED': 41
}

# Fixed RandomSeed
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED'])  # Seed 고정

# Data Pre-processing
df = pd.read_csv('./train.csv')

train_len = int(len(df) * 0.8)
train_df = df.iloc[:train_len].reset_index(drop=True)
val_df = df.iloc[train_len:].reset_index(drop=True)

train_label_vec = train_df.iloc[:, 2:].values.astype(np.float32)
val_label_vec = val_df.iloc[:, 2:].values.astype(np.float32)

# 라벨 값을 로그 스케일로 변환
train_label_vec = np.log1p(train_label_vec)
val_label_vec = np.log1p(val_label_vec)

CFG['label_size'] = train_label_vec.shape[1]

# CustomDataset
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list=None, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms

    def __getitem__(self, index):
        img_path = self.img_path_list[index]

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transforms is not None:
            image = self.transforms(image=image)['image']

        if self.label_list is not None:
            label = self.label_list[index]
            label = torch.tensor(label, dtype=torch.float32)
            return image, label
        else:
            return image

    def __len__(self):
        return len(self.img_path_list)

train_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

test_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

train_dataset = CustomDataset(train_df['path'].values, train_label_vec, train_transform)
train_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val_df['path'].values, val_label_vec, test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

# Model Define
class BaseModel(nn.Module):
    def __init__(self, gene_size=CFG['label_size']):
        super(BaseModel, self).__init__()
        # ViT 모델 로드 (ImageNet-21k로 사전 학습된 가중치 사용)
        self.backbone = timm.create_model('vit_base_patch16_224', pretrained=True)
        # 분류 헤드를 제거하고 특징 추출기만 사용
        self.backbone.head = nn.Identity()
        # 백본의 출력 특징 수
        num_features = self.backbone.num_features
        # 회귀를 위한 새로운 출력층 추가
        self.regressor = nn.Linear(num_features, gene_size)

    def forward(self, x):
        x = self.backbone(x)
        x = self.regressor(x)
        return x

# Pearson Correlation Coefficient Loss 정의
class PearsonLoss(nn.Module):
    def __init__(self):
        super(PearsonLoss, self).__init__()

    def forward(self, y_pred, y_true):
        # y_pred와 y_true의 평균을 계산
        y_pred_mean = torch.mean(y_pred, dim=0)
        y_true_mean = torch.mean(y_true, dim=0)

        # 분자 계산
        vx = y_pred - y_pred_mean
        vy = y_true - y_true_mean
        numerator = torch.sum(vx * vy, dim=0)

        # 분모 계산
        denominator = torch.sqrt(torch.sum(vx ** 2, dim=0)) * torch.sqrt(torch.sum(vy ** 2, dim=0))

        # 각 차원별 Pearson 상관계수 계산
        correlation = numerator / (denominator + 1e-8)

        # 상관계수의 평균을 구하고 손실로 사용
        loss = 1 - torch.mean(correlation)

        return loss

# Train
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    # criterion을 PearsonLoss로 변경
    criterion = PearsonLoss().to(device)

    best_loss = float('inf')
    best_model = None

    for epoch in range(1, CFG['EPOCHS'] + 1):
        model.train()
        train_loss = 0.0
        total_batches = 0

        train_bar = tqdm(train_loader, desc=f"Epoch {epoch} Training", leave=False)
        for imgs, labels in train_bar:
            imgs = imgs.float().to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            output = model(imgs)
            loss = criterion(output, labels)

            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            total_batches += 1

            train_bar.set_postfix(loss=loss.item())

        avg_train_loss = train_loss / total_batches
        avg_val_loss = validation(model, criterion, val_loader, device, epoch)

        print(f'Epoch [{epoch}], Train Loss: [{avg_train_loss:.5f}], Val Loss: [{avg_val_loss:.5f}]')

        if scheduler is not None:
            scheduler.step(avg_val_loss)

        if best_loss > avg_val_loss:
            best_loss = avg_val_loss
            best_model = model.state_dict()

    model.load_state_dict(best_model)
    return model

# valid
def validation(model, criterion, val_loader, device, epoch):
    model.eval()
    val_loss = 0.0
    total_batches = 0

    val_bar = tqdm(val_loader, desc=f"Epoch {epoch} Validation", leave=False)
    with torch.no_grad():
        for imgs, labels in val_bar:
            imgs = imgs.float().to(device)
            labels = labels.to(device)

            pred = model(imgs)
            loss = criterion(pred, labels)

            val_loss += loss.item()
            total_batches += 1

            val_bar.set_postfix(loss=loss.item())

    avg_val_loss = val_loss / total_batches
    return avg_val_loss

# Run!!
model = BaseModel()
model.to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2,
                                                       threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

# Inference
test = pd.read_csv('./test.csv')

test_dataset = CustomDataset(test['path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(test_loader):
            imgs = imgs.to(device).float()
            pred = model(imgs)
            preds.append(pred.detach().cpu())

    preds = torch.cat(preds).numpy()
    # 예측값에 역로그 변환 적용
    preds = np.expm1(preds)
    return preds

preds = inference(infer_model, test_loader, device)

# Submission
submit = pd.read_csv('./sample_submission.csv')
submit.iloc[:, 1:] = np.array(preds).astype(np.float32)
submit.to_csv('./baseline_submit.csv', index=False)


Epoch [1], Train Loss: [0.79914], Val Loss: [0.74077]


Epoch [2], Train Loss: [0.71691], Val Loss: [0.71670]


Epoch [3], Train Loss: [0.68170], Val Loss: [0.69151]


Epoch [4], Train Loss: [0.66350], Val Loss: [0.70101]


Epoch [5], Train Loss: [0.64365], Val Loss: [0.69026]


100%|██████████| 72/72 [00:15<00:00,  4.80it/s]
